# M2 Force balance system test through a bump sequence - tangent actuators

This Jupyter notebook is to run the bump test of M2 to verify that the M2 force balance system works as expected in closed-loop.
The A2 tangent actuator is issued with a fixed force of 300 N with pull/push (+/-) movement one by one.

## Import Modules

In [1]:
import asyncio
import numpy as np
from datetime import datetime
from lsst.ts import salobj
from lsst.ts.m2com import NUM_ACTUATOR, NUM_TANGENT_LINK

## Declaration of User-defined Functions

In [2]:
async def injectForce_tangent_actuator(csc, actuators, force, sleep_time=5):
    """inject a force to the axial actuator.
    
    Parameters
    ----------
    csc : lsst.ts.salobj.remote.Remote
        Remote object of the M2 CSC.
    actuators : list of actuators.
    force : list of forces
        Force to apply (Newton).
    sleep_time : float, optional
        Sleep time. (the default is 5.0)
    """

    forces = [0.] * NUM_TANGENT_LINK
    
    for idx in range(len(actuators)):
        print(f"idx: {idx}.")
        index = actuators[idx]
        forces[index] = force
        print(f"Apply the force: {force} N. to actuator: {index}")
    await csc.cmd_applyForces.set_start(tangent=forces)
    await asyncio.sleep(sleep_time)


## Prepare the M2 CSC and put to Enabled state

In [3]:
domain = salobj.Domain()
m2 = salobj.Remote(domain, "MTM2")
await m2.start_task
await m2.cmd_setLogLevel.set_start(level=10)

In [14]:
# get status

state = m2.evt_summaryState.get()
print(state)

private_revCode: 0ebebdcc, private_sndStamp: 1688613950.199437, private_rcvStamp: 1688613950.1996267, private_seqNum: 8, private_identity: MTM2, private_origin: 56731, summaryState: 3


In [5]:
# Standby  -->  Disable
await m2.cmd_start.set_start(timeout=60)

In [7]:
# Disable  -->  Enabled
await m2.cmd_enable.set_start(timeout=200)

In [13]:
# Enabled  -->  Disable
await m2.cmd_disable.set_start(timeout=30)

In [ ]:
#Disable  -->  Standby
await m2.cmd_standby.set_start(timeout=30)

In [ ]:
#Fault --> Standby
await m2.cmd_standby.set_start(timeout=30)

## Do the force actuator limit Test

In [9]:
# set actuator number
actuators = [1]
n_act = 1#NUM_TANGENT_LINK


# set tangent actuator forces
f_Delta = 300
forces = f_Delta

In [11]:
time_start = datetime.now()
print(f"UTC time to is {time_start} now.")

await m2.cmd_resetForceOffsets.set_start()
await asyncio.sleep(5)


#apply force to A2
await injectForce_tangent_actuator(m2, actuators, f_Delta, sleep_time=60)
await asyncio.sleep(10)
await m2.cmd_resetForceOffsets.set_start()
await asyncio.sleep(10)

await injectForce_tangent_actuator(m2, actuators, -f_Delta, sleep_time=60)
await asyncio.sleep(10)
await m2.cmd_resetForceOffsets.set_start()
await asyncio.sleep(10)

time_end = datetime.now()
print(f"UTC time to is {time_end} now.")

UTC time to is 2023-07-06 03:20:44.887822 now.
idx: 0.
Apply the force: 300 N. to actuator: 1
idx: 0.
Apply the force: -300 N. to actuator: 1
UTC time to is 2023-07-06 03:23:32.407059 now.
